In [1]:
import os
import sys
import time

from matplotlib import animation
from matplotlib.colorbar import Colorbar
from matplotlib.gridspec import GridSpec
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import interpolate, signal, stats, fftpack

from pyedflib import highlevel

#### Change this path to point to your code

In [2]:
sys.path.append(r'C:\Users\lesliec\code')

In [3]:
from tbd_eeg.tbd_eeg.data_analysis.eegutils import EEGexp

In [4]:
%matplotlib notebook

In [5]:
edf_folder = r"E:\GAT1_EEG_pilot\edf_files"

Load .csv with list of experiments

In [6]:
exp_file = r"C:\Users\lesliec\OneDrive - Allen Institute\data\GAT1KO_pilot_exps_all.csv"
subject_df = pd.read_csv(exp_file, converters={'mouse': str})

In [7]:
subject_df.head()

,genotype,mouse,sex,experiment,sweep_states,bad_chs,data_loc,notes
0,GAT1-KO,645606,Male,signal_test_2022-11-18_14-49-19,awake,none,E:\GAT1_EEG_pilot\mouse645606\signal_test_2022...,NaN
1,GAT1-KO,645606,Male,spontEEG1_2022-11-22_17-12-47,awake,none,E:\GAT1_EEG_pilot\mouse645606\spontEEG1_2022-1...,NaN
2,GAT1-KO,645606,Male,EEGNPXspont_estim_2022-12-20_12-26-39,awake,none,E:\GAT1_EEG_pilot\mouse645606\EEGNPXspont_esti...,NaN
3,GAT1-KO,644565,Female,signal_test_2022-11-30_15-41-47,awake,none,E:\GAT1_EEG_pilot\mouse644565\signal_test_2022...,NaN
4,GAT1-KO,644565,Female,spontEEG1_2022-12-07_16-07-14,awake,none,E:\GAT1_EEG_pilot\mouse644565\spontEEG1_2022-1...,NaN


### Loop through experiments to create .edf file for each

In [8]:
edf_ch = 18

for ii, rowi in subject_df.iterrows():
    exp = EEGexp(rowi.data_loc, preprocess=False, make_stim_csv=False)
    exp_tag = exp.experiment_folder[exp.experiment_folder.find('mouse')+12:exp.experiment_folder.find(str(exp.date.year))-1]
    print('{} - {}'.format(exp.mouse, exp_tag))
    
    eeg_data, eeg_ts = exp.load_eegdata()
    eeg_data = eeg_data * 1E-3 # convert to mV
    eeg_chs = np.arange(0, eeg_data.shape[1])
    
    ## Name the edf file ##
    edf_name = 'mouse' + exp.mouse + '_' + exp.date.strftime('%Y-%m-%d') + '_' + exp_tag + '_ch' + str(edf_ch) + '.edf'
    print(' {}'.format(edf_name))
    edf_file1 = os.path.join(exp.data_folder, edf_name)
    edf_file2 = os.path.join(edf_folder, edf_name)
    
    ## Set up the edf file ##
    signal_headers = highlevel.make_signal_headers(
        ['ch{:d}'.format(edf_ch)], sample_frequency=exp.ephys_params['EEG']['sample_rate']
    )
    header = highlevel.make_header(patientname='mouse' + exp.mouse, gender=rowi.sex)
    
    ## Make and save the edf file (twice) ##
    highlevel.write_edf(edf_file1, eeg_data[:, edf_ch][np.newaxis, :], signal_headers, header)
    highlevel.write_edf(edf_file2, eeg_data[:, edf_ch][np.newaxis, :], signal_headers, header)
    print(' Saved!\n')

Experiment type: sensory stimulation
645606 - signal_test
 mouse645606_2022-11-18_signal_test_ch18.edf
 Saved!

No *stim* files were found.
Experiment type: spontaneous
645606 - spontEEG1
 mouse645606_2022-11-22_spontEEG1_ch18.edf
 Saved!

Experiment type: electrical stimulation
645606 - EEGNPXspont_estim
 mouse645606_2022-12-20_EEGNPXspont_estim_ch18.edf
 Saved!

Experiment type: sensory stimulation
644565 - signal_test
 mouse644565_2022-11-30_signal_test_ch18.edf
 Saved!

No *stim* files were found.
Experiment type: spontaneous
644565 - spontEEG1
 mouse644565_2022-12-07_spontEEG1_ch18.edf
 Saved!

Experiment type: electrical stimulation
644565 - EEGNPXspont_estim
 mouse644565_2022-12-22_EEGNPXspont_estim_ch18.edf
 Saved!

Experiment type: sensory stimulation
672785 - visual
 mouse672785_2023-06-27_visual_ch18.edf
 Saved!

Experiment type: electrical and sensory stimulation
672785 - EEGNPXspont_estim
 mouse672785_2023-07-05_EEGNPXspont_estim_ch18.edf
 Saved!

Experiment type: sensory 

## Practice with one subject

### Load experiment

In [5]:
recfolder = r'E:\GAT1_EEG_pilot\mouse645606\signal_test_2022-11-18_14-49-19\experiment1\recording1'
# exp = EEGexp(recfolder, preprocess=True, make_stim_csv=True)
exp = EEGexp(recfolder, preprocess=False, make_stim_csv=False)

Experiment type: sensory stimulation


In [6]:
exp_tag = exp.experiment_folder[exp.experiment_folder.find('mouse')+12:exp.experiment_folder.find(str(exp.date.year))-1]
print(exp_tag)

signal_test


In [7]:
GAT_mice = {
    '645606': 'Male',
    '644565': 'Female',
    '672785': 'Male',
    '672789': 'Female',
}

### Load EEG data

In [8]:
eeg_data, eeg_ts = exp.load_eegdata()
eeg_data = eeg_data * 1E-3 # convert to mV
eeg_chs = np.arange(0, eeg_data.shape[1])

bad_chs = []
GOOD_CHS = np.array([x for x in eeg_chs if x not in bad_chs])

In [9]:
print(eeg_ts[0])
print(eeg_ts[-1])

33.12216963591622
944.3795366517253


In [21]:
(944-33)/60

15.183333333333334

### Package one ch as .edf

Following example here: https://github.com/holgern/pyedflib/

In [10]:
edf_ch = 18

edf_name = 'mouse' + exp.mouse + '_' + exp.date.strftime('%Y-%m-%d') + '_' + exp_tag + '_ch' + str(edf_ch) + '.edf'
print(edf_name)
edf_file = os.path.join(exp.data_folder, edf_name)
# print(edf_file)

mouse645606_2022-11-18_signal_test_ch18.edf


In [11]:
channel_names = ['ch{:d}'.format(edf_ch)]

In [12]:
signal_headers = highlevel.make_signal_headers(channel_names, sample_frequency=exp.ephys_params['EEG']['sample_rate'])

In [13]:
header = highlevel.make_header(patientname='mouse' + exp.mouse, gender=GAT_mice[exp.mouse])

In [20]:
highlevel.write_edf(edf_file, eeg_data[:, edf_ch][np.newaxis, :], signal_headers, header)

True

# This works!!